In [1]:
# ════════════════════════════════════════════════════════════════
# LEVEL 5: BUSINESS INTELLIGENCE & CLOUD DEPLOYMENT
# Climate Risk Insurance Project
# ════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import pickle

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

print("="*70)
print("📊 LEVEL 5: BUSINESS INTELLIGENCE & CLOUD DEPLOYMENT")
print("="*70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("="*70)

# ════════════════════════════════════════════════════════════════
# STEP 5.0: LOAD DATA & MODELS FROM LEVEL 4
# ════════════════════════════════════════════════════════════════

print("\n📂 STEP 5.0: Loading data and models from Level 4...")

# Load processed data
df_model = pd.read_csv('data/processed/final_model_data.csv')
print(f"✅ Loaded data: {len(df_model):,} policies")

# Load trained models
with open('outputs/models/xgb_classification_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
print("✅ Loaded XGBoost classification model")

with open('outputs/models/xgb_regression_model.pkl', 'rb') as f:
    xgb_reg = pickle.load(f)
print("✅ Loaded XGBoost regression model")

# Load feature list
with open('outputs/models/feature_list.txt', 'r') as f:
    content = f.read()
    # Extract feature names from the file
    feature_lines = [line.strip() for line in content.split('\n') if line.strip() and line[0].isdigit()]
    model_features = [line.split('. ')[1] for line in feature_lines]

print(f"✅ Loaded {len(model_features)} features")

# ════════════════════════════════════════════════════════════════
# STEP 5A.1: PREPARE DATA FOR POWERBI
# ════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📊 STEP 5A.1: POWERBI DATA PREPARATION")
print("="*70)

# Create PowerBI directory
os.makedirs('outputs/powerbi', exist_ok=True)

# Select relevant columns for PowerBI
powerbi_columns = [
    'policy_id', 'state', 'county', 'latitude', 'longitude',
    'property_value', 'property_age', 'building_type',
    'RISK_SCORE', 'current_annual_premium', 'coverage_amount', 
    'deductible', 'policy_start_year',
    'num_claims_5yr', 'total_claim_amount_5yr'
]

# Check which columns exist
available_cols = [col for col in powerbi_columns if col in df_model.columns]
powerbi_data = df_model[available_cols].copy()

print(f"\n📋 Creating PowerBI dataset...")
print(f"   Original columns: {len(available_cols)}")

# Add calculated columns for PowerBI
powerbi_data['Year'] = 2024
powerbi_data['policy_age'] = 2024 - powerbi_data['policy_start_year']

# Risk categories
powerbi_data['Risk_Category'] = pd.cut(
    powerbi_data['RISK_SCORE'],
    bins=[0, 20, 40, 60, 80, 100],
    labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High']
)

# Premium categories
powerbi_data['Premium_Category'] = pd.cut(
    powerbi_data['current_annual_premium'],
    bins=[0, 1500, 2500, 3500, 5000, 10000],
    labels=['< $1.5K', '$1.5K-$2.5K', '$2.5K-$3.5K', '$3.5K-$5K', '> $5K']
)

# Claim status
powerbi_data['has_claim'] = (powerbi_data['num_claims_5yr'] > 0).astype(int)
powerbi_data['Claim_Status'] = powerbi_data['has_claim'].map({0: 'No Claim', 1: 'Has Claim'})

# Coverage ratio
powerbi_data['coverage_ratio'] = powerbi_data['coverage_amount'] / powerbi_data['property_value']

# Loss ratio
powerbi_data['Loss_Ratio'] = (
    powerbi_data['total_claim_amount_5yr'] / 
    (powerbi_data['current_annual_premium'] * 5)
).fillna(0).clip(0, 5)

print(f"✅ Enhanced dataset: {powerbi_data.shape[1]} columns")

# Prepare features for model predictions
# ════════════════════════════════════════════════════════════════
# Recreate Missing Features for Model Predictions
# ════════════════════════════════════════════════════════════════

print("\n🔧 Recreating engineered features for predictions...")

# Create composite_risk if missing
if 'composite_risk' not in df_model.columns:
    df_model['composite_risk'] = df_model['RISK_SCORE'].copy()
    if 'CFLD_RISKS' in df_model.columns:
        df_model['composite_risk'] += df_model['CFLD_RISKS'].replace(-9999, 0) * 0.3
    if 'WFIR_RISKS' in df_model.columns:
        df_model['composite_risk'] += df_model['WFIR_RISKS'].replace(-9999, 0) * 0.3

# Create risk indicators if missing
if 'is_high_risk' not in df_model.columns:
    df_model['is_high_risk'] = (df_model['RISK_SCORE'] > 60).astype(int)

if 'is_very_high_risk' not in df_model.columns:
    df_model['is_very_high_risk'] = (df_model['RISK_SCORE'] > 80).astype(int)

# Create property features if missing
if 'age_risk_interaction' not in df_model.columns:
    df_model['age_risk_interaction'] = df_model['property_age'] * df_model['RISK_SCORE']

if 'is_single_family' not in df_model.columns:
    df_model['is_single_family'] = (df_model['building_type'] == 'Single_Family').astype(int)

if 'is_commercial' not in df_model.columns:
    df_model['is_commercial'] = (df_model['building_type'] == 'Commercial').astype(int)

# Create policy features if missing
if 'coverage_ratio' not in df_model.columns:
    df_model['coverage_ratio'] = df_model['coverage_amount'] / df_model['property_value']

if 'policy_age' not in df_model.columns:
    df_model['policy_age'] = 2024 - df_model['policy_start_year']

if 'has_high_deductible' not in df_model.columns:
    df_model['has_high_deductible'] = (df_model['deductible'] >= 5000).astype(int)

# Create geographic features if missing
if 'is_high_risk_state' not in df_model.columns:
    high_risk_states = ['CA', 'FL', 'TX', 'LA', 'NY', 'NJ']
    df_model['is_high_risk_state'] = df_model['state'].isin(high_risk_states).astype(int)

# Create construction features if missing
if 'construction_type' in df_model.columns:
    if 'is_wood_construction' not in df_model.columns:
        df_model['is_wood_construction'] = (df_model['construction_type'] == 'Wood').astype(int)
    if 'is_fire_resistant' not in df_model.columns:
        df_model['is_fire_resistant'] = df_model['construction_type'].isin(['Steel', 'Concrete']).astype(int)

print("✅ All required features recreated")

# Verify all features exist
missing_features = [f for f in model_features if f not in df_model.columns]
if missing_features:
    print(f"⚠️  Still missing: {missing_features}")
    # Use only available features
    available_features = [f for f in model_features if f in df_model.columns]
    print(f"✅ Using {len(available_features)} available features")
    X_features = df_model[available_features].copy()
else:
    print(f"✅ All {len(model_features)} features available")
    X_features = df_model[model_features].copy()
print("✅ Model predictions added")

# ════════════════════════════════════════════════════════════════
# Create Summary Tables
# ════════════════════════════════════════════════════════════════

print("\n📊 Creating summary tables...")

# State summary
state_summary = powerbi_data.groupby('state').agg({
    'policy_id': 'count',
    'property_value': 'sum',
    'current_annual_premium': ['sum', 'mean'],
    'RISK_SCORE': 'mean',
    'num_claims_5yr': 'sum',
    'total_claim_amount_5yr': 'sum'
}).round(2)

state_summary.columns = [
    'Total_Policies', 'Total_Property_Value', 'Total_Premium', 
    'Avg_Premium', 'Avg_Risk_Score', 'Total_Claims', 'Total_Claim_Amount'
]
state_summary = state_summary.reset_index()
state_summary['Claim_Rate'] = (
    state_summary['Total_Claims'] / (state_summary['Total_Policies'] * 5) * 100
).round(1)
state_summary['Loss_Ratio'] = (
    state_summary['Total_Claim_Amount'] / (state_summary['Total_Premium'] * 5) * 100
).round(1)

print(f"✅ State summary: {len(state_summary)} states")

# Risk category summary
risk_summary = powerbi_data.groupby('Risk_Category').agg({
    'policy_id': 'count',
    'current_annual_premium': 'mean',
    'num_claims_5yr': 'sum',
    'total_claim_amount_5yr': 'sum'
}).round(2)
risk_summary.columns = ['Policies', 'Avg_Premium', 'Total_Claims', 'Claim_Amount']
risk_summary = risk_summary.reset_index()
risk_summary['Claim_Rate'] = (
    risk_summary['Total_Claims'] / (risk_summary['Policies'] * 5) * 100
).round(1)

print(f"✅ Risk summary: {len(risk_summary)} categories")

# ════════════════════════════════════════════════════════════════
# Save Files
# ════════════════════════════════════════════════════════════════

print("\n💾 Saving PowerBI files...")

powerbi_data.to_csv('outputs/powerbi/climate_risk_data.csv', index=False)
print("✅ Main data: outputs/powerbi/climate_risk_data.csv")

state_summary.to_csv('outputs/powerbi/state_summary.csv', index=False)
print("✅ State summary: outputs/powerbi/state_summary.csv")

risk_summary.to_csv('outputs/powerbi/risk_summary.csv', index=False)
print("✅ Risk summary: outputs/powerbi/risk_summary.csv")

# Data dictionary
data_dictionary = f"""
CLIMATE RISK INSURANCE - POWERBI DATA DICTIONARY
================================================

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

MAIN DATASET: climate_risk_data.csv ({len(powerbi_data):,} records)
-----------------------------------------------------------

IDENTIFIERS:
- policy_id: Unique policy identifier

LOCATION:
- state: US state (2-letter code)
- county: County name
- latitude: Geographic latitude
- longitude: Geographic longitude

PROPERTY:
- property_value: Property value ($)
- property_age: Age in years
- building_type: Single_Family/Multi_Family/Commercial

RISK:
- RISK_SCORE: FEMA risk (0-100, higher = riskier)
- Risk_Category: Very Low/Low/Moderate/High/Very High

FINANCIAL:
- current_annual_premium: Current premium ($)
- coverage_amount: Coverage ($)
- deductible: Deductible amount ($)
- Premium_Category: Premium bracket
- coverage_ratio: Coverage/Property Value

CLAIMS:
- num_claims_5yr: Claims in past 5 years
- total_claim_amount_5yr: Total claims ($)
- has_claim: 0 or 1
- Claim_Status: "No Claim" or "Has Claim"
- Loss_Ratio: Claims/Premiums (capped at 5.0)

ML PREDICTIONS:
- Predicted_Claim_Probability: Claim risk (0-1)
- Predicted_Premium: Recommended premium ($)
- Premium_Gap: Predicted - Current ($)
- Is_Underpriced: Underpriced or Fair/Overpriced

STATE SUMMARY: state_summary.csv ({len(state_summary)} states)
------------------------------------------------------
Aggregated metrics by state

RISK SUMMARY: risk_summary.csv ({len(risk_summary)} categories)
---------------------------------------------------
Aggregated metrics by risk level
"""

with open('outputs/powerbi/DATA_DICTIONARY.txt', 'w') as f:
    f.write(data_dictionary)

print("✅ Data dictionary: outputs/powerbi/DATA_DICTIONARY.txt")

# ════════════════════════════════════════════════════════════════
# Summary
# ════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("✅ POWERBI DATA PREPARATION COMPLETE!")
print("="*70)
print(f"\n📊 Dataset Summary:")
print(f"   Records:     {len(powerbi_data):,}")
print(f"   Columns:     {powerbi_data.shape[1]}")
print(f"   States:      {powerbi_data['state'].nunique()}")
print(f"   Total Value: ${powerbi_data['property_value'].sum()/1e9:.2f}B")

print(f"\n📂 Files ready for PowerBI:")
print(f"   1. climate_risk_data.csv     ({len(powerbi_data):,} rows)")
print(f"   2. state_summary.csv         ({len(state_summary)} states)")
print(f"   3. risk_summary.csv          ({len(risk_summary)} categories)")
print(f"   4. DATA_DICTIONARY.txt       (column guide)")

print("\n🎯 Next Steps:")
print("   1. Open PowerBI Desktop")
print("   2. Import climate_risk_data.csv")
print("   3. Follow dashboard creation guide")

print("\n" + "="*70)

📊 LEVEL 5: BUSINESS INTELLIGENCE & CLOUD DEPLOYMENT
Analysis Date: 2026-02-14 10:29

📂 STEP 5.0: Loading data and models from Level 4...
✅ Loaded data: 15,000 policies
✅ Loaded XGBoost classification model
✅ Loaded XGBoost regression model
✅ Loaded 12 features

📊 STEP 5A.1: POWERBI DATA PREPARATION

📋 Creating PowerBI dataset...
   Original columns: 15
✅ Enhanced dataset: 23 columns

🔧 Recreating engineered features for predictions...
✅ All required features recreated
✅ All 12 features available
✅ Model predictions added

📊 Creating summary tables...
✅ State summary: 56 states
✅ Risk summary: 5 categories

💾 Saving PowerBI files...
✅ Main data: outputs/powerbi/climate_risk_data.csv
✅ State summary: outputs/powerbi/state_summary.csv
✅ Risk summary: outputs/powerbi/risk_summary.csv
✅ Data dictionary: outputs/powerbi/DATA_DICTIONARY.txt

✅ POWERBI DATA PREPARATION COMPLETE!

📊 Dataset Summary:
   Records:     15,000
   Columns:     23
   States:      56
   Total Value: $4.83B

📂 Files rea